In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS
from langchain import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.document_transformers import LongContextReorder
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_ollama import OllamaLLM


In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH00-RAG-Chatbot")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH00-RAG-Chatbot


In [4]:
OLLAMA_BASE_URL = "http://localhost:11434" # Ollama 서버 주소
OLLAMA_CHAT_MODEL = "ollama-ko-0710:latest" # Ollama에서 실행중인 모델 이름

In [5]:
def get_split_docs():
    # 1. 텍스트 청크 분할 (chunk_size=1000, chunk_overlap=50)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

    # 2. Text loader
    loader = TextLoader("./data/text.txt", encoding="utf-8")
    return loader.load_and_split(text_splitter)

In [6]:
def get_retriever(split_docs): 
    model_name = "intfloat/multilingual-e5-large-instruct"
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda"},  # cuda, cpu
        encode_kwargs={"normalize_embeddings": True},
    )
    db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.25, "fetch_k": 20}
    )
    return retriever

In [7]:
def get_bm25_retriever(split_docs):
    bm25_retriever = BM25Retriever.from_documents(
        split_docs
    )
    bm25_retriever.k = 10  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.
    return bm25_retriever


In [8]:
def get_reranker(retriever):
    reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
    compressor = CrossEncoderReranker(model=reranker, top_n=10)
    return ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )

In [10]:
def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    return reordered_docs

In [11]:
def get_prompt():
    template = """당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.
    You are a helpful AI assistant, you'll need to answer users' queries in a friendly and accurate manner.
    모든 대답은 반드시 한국말로 대답해주세요.
    
    문서 내용:
    {context}
    질문: {question}
    답변:"""
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    return prompt

In [12]:
def get_multi_query_retriever(retriever):
    # 다중 질의어 생성
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=retriever,
    llm=llm,
    )
    return multiquery_retriever

In [ ]:
def get_esenmble_retriever(retriever1, retriever2):
    ensemble_retriever = EnsembleRetriever(
        retrievers=[retriever1, retriever2],
        weights=[0.7, 0.5],  # 각 리트리버의 가중치를 설정합니다.
        k=10  # 최종적으로 반환할 문서의 개수를 설정합니다.
    )
    return ensemble_retriever

## ReRank+ ReOrder  추가 버전

In [ ]:
def general_llm():

    import logging
    logging.basicConfig()
    logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

    split_docs = get_split_docs()

    faiss_retriever = get_retriever(split_docs)
    
    bm25_retriever = get_bm25_retriever(split_docs)

    faiss_multi_retriever = get_multi_query_retriever(faiss_retriever)

    bm25_multi_retriever = get_multi_query_retriever(bm25_retriever)

    esenmble_retriever = get_esenmble_retriever(faiss_multi_retriever, bm25_multi_retriever)

    compression_retriever = get_reranker(esenmble_retriever)
    
    prompt= get_prompt()
    
    
    # 7. Ollama LLM 초기화
    # llm = OllamaLLM(model=OLLAMA_CHAT_MODEL, base_url=OLLAMA_BASE_URL,temperature=1, max_tokens=1024)
    llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0.7, max_tokens=1024)
    chain = (
    {
        "context": itemgetter("question")
        | compression_retriever
        | RunnableLambda(reorder_documents),  # 질문을 기반으로 문맥을 검색합니다.
        "question": itemgetter("question"),  # 질문을 추출합니다.
    }
    | prompt  # 프롬프트 템플릿에 값을 전달합니다.
    | llm # 언어 모델에 프롬프트를 전달합니다.
    | StrOutputParser()  # 모델의 출력을 문자열로 파싱합니다.
    )

    answer = chain.invoke({"question": "삼성전자 하반기 실적 전망은?"})
    return answer

In [ ]:
general_llm()



INFO:langchain.retrievers.multi_query:Generated queries: ['삼성전자의 2023년 하반기 실적 예측은 어떻게 되나요?  ', '올해 하반기 삼성전자의 재무 성과에 대한 전망은 무엇인가요?  ', '삼성전자의 하반기 실적에 대한 분석이나 예측이 있나요?']
INFO:langchain.retrievers.multi_query:Generated queries: ['삼성전자의 2023년 하반기 실적 예측은 어떻게 되나요?  ', '올해 하반기 삼성전자의 재무 성과에 대한 전망은 무엇인가요?  ', '삼성전자의 하반기 실적에 대한 분석이나 예측이 있나요?']


'삼성전자의 하반기 실적 전망은 긍정적입니다. NH투자증권 류영호 연구원은 파운드리 부문에서 테슬라와 애플 등 주요 고객사를 확보했으며, 10나노급 6세대(1c) 수율 개선과 하반기 엔비디아 고대역폭 메모리(HBM) 공급 기대감이 주가에 긍정적인 영향을 미치고 있다고 밝혔습니다. 또한 3분기 영업이익은 약 9조원으로, 전년 동기 대비 1.9% 감소하나 직전 분기 대비 91.9% 증가할 것으로 전망되고 있습니다.\n\n미래에셋증권은 올해와 내년 삼성전자 영업이익 전망치를 각각 2.9%, 5.3% 상향 조정했으며, 목표 주가도 8만8000원에서 9만6000원으로 9% 올렸습니다. 이는 내년 메모리 반도체 공급 부족 상황과 차세대 고대역폭 메모리(HBM) 생산 집중에 따른 가격 상승 기대 때문입니다.\n\n종합하면, 삼성전자는 하반기에도 파운드리와 고대역폭 메모리 부문을 중심으로 실적 개선과 주가 상승의 긍정적 흐름이 이어질 것으로 예상됩니다.'

: 